# Multi-Hop QA Dataset Embedding & Graph Construction

## 목적
HotpotQA와 2WikiMultiHopQA 데이터셋에 대해 Step별(250, 500, 750, 1000) 임베딩 및 그래프를 구축

## 출력 경로
- `_hippo_rag_MHQA_CL/hotpotqa/step_{N}/`
- `_hippo_rag_MHQA_CL/2wikimultihopqa/step_{N}/`

## 입력 데이터
- **Dataset**: `reproduce/dataset/{dataset}.json`, `reproduce/dataset/{dataset}_corpus.json`
- **OpenIE**: `outputs/hipporag_hotpotqa/openie_results_ner_gpt-4o-mini.json` 등

In [1]:
# ============================================================
# 1. Imports & Configuration
# ============================================================
import os
import json
import hashlib
import gc
import pickle
from pathlib import Path
from typing import List, Dict, Any, Set, Tuple
from dataclasses import dataclass

import numpy as np
import torch

# HippoRAG 경로 설정 (모듈이 src/ 안에 있음)
import sys
sys.path.append("/NAS/minyeol/hippoRAG/src")

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,3"

from hipporag.HippoRAG import HippoRAG, compute_mdhash_id
from hipporag.utils.config_utils import BaseConfig


/home/minyeol/.conda/envs/hipporagenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026-02-02 13:53:56,037	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


✓ 모듈 임포트 완료


In [2]:
# ============================================================
# 2. Configuration
# ============================================================
"""
데이터셋별 설정
- 경로, Step 값, LLM/Embedding 모델 설정

주의: 노트북이 _hippo_rag_MHQA_CL/ 폴더에 있으므로 절대경로 사용
"""

# 프로젝트 루트 경로 (절대경로)
PROJECT_ROOT = "/NAS/minyeol/hippoRAG"

@dataclass
class DatasetConfig:
    """데이터셋 설정"""
    name: str                    # 데이터셋 이름 (hotpotqa, 2wikimultihopqa)
    samples_path: str            # 샘플 JSON 경로
    corpus_path: str             # 코퍼스 JSON 경로
    openie_path: str             # OpenIE 결과 JSON 경로
    output_base_dir: str         # 출력 베이스 디렉토리
    
    @property
    def output_dir(self) -> str:
        return f"{self.output_base_dir}/{self.name}"


# 데이터셋 설정 (절대경로 사용)
DATASETS = {
    "hotpotqa": DatasetConfig(
        name="hotpotqa",
        samples_path=f"{PROJECT_ROOT}/reproduce/dataset/hotpotqa.json",
        corpus_path=f"{PROJECT_ROOT}/reproduce/dataset/hotpotqa_corpus.json",
        openie_path=f"{PROJECT_ROOT}/outputs/hotpotqa/openie_results_ner_gpt-4o-mini.json",
        output_base_dir=f"{PROJECT_ROOT}/_hippo_rag_MHQA_CL",
    ),
    "2wikimultihopqa": DatasetConfig(
        name="2wikimultihopqa",
        samples_path=f"{PROJECT_ROOT}/reproduce/dataset/2wikimultihopqa.json",
        corpus_path=f"{PROJECT_ROOT}/reproduce/dataset/2wikimultihopqa_corpus.json",
        openie_path=f"{PROJECT_ROOT}/outputs/2wikimultihopqa/openie_results_ner_gpt-4o-mini.json",
        output_base_dir=f"{PROJECT_ROOT}/_hippo_rag_MHQA_CL",
    ),
}

# 공통 설정
STEPS = [250, 500, 750, 1000]
LLM_NAME = "gpt-4o-mini"
EMBEDDING_MODEL_NAME = "nvidia/NV-Embed-v2"

print("✓ 설정 완료")
print(f"  PROJECT_ROOT: {PROJECT_ROOT}")
print(f"  Steps: {STEPS}")
print(f"  Datasets: {list(DATASETS.keys())}")

# 경로 검증
print("\n경로 검증:")
for name, config in DATASETS.items():
    samples_ok = "✓" if os.path.exists(config.samples_path) else "❌"
    corpus_ok = "✓" if os.path.exists(config.corpus_path) else "❌"
    openie_ok = "✓" if os.path.exists(config.openie_path) else "❌"
    print(f"  [{name}]")
    print(f"    samples: {samples_ok} {config.samples_path}")
    print(f"    corpus:  {corpus_ok} {config.corpus_path}")
    print(f"    openie:  {openie_ok} {config.openie_path}")

✓ 설정 완료
  PROJECT_ROOT: /NAS/minyeol/hippoRAG
  Steps: [250, 500, 750, 1000]
  Datasets: ['hotpotqa', '2wikimultihopqa']

경로 검증:
  [hotpotqa]
    samples: ✓ /NAS/minyeol/hippoRAG/reproduce/dataset/hotpotqa.json
    corpus:  ✓ /NAS/minyeol/hippoRAG/reproduce/dataset/hotpotqa_corpus.json
    openie:  ✓ /NAS/minyeol/hippoRAG/outputs/hotpotqa/openie_results_ner_gpt-4o-mini.json
  [2wikimultihopqa]
    samples: ✓ /NAS/minyeol/hippoRAG/reproduce/dataset/2wikimultihopqa.json
    corpus:  ✓ /NAS/minyeol/hippoRAG/reproduce/dataset/2wikimultihopqa_corpus.json
    openie:  ✓ /NAS/minyeol/hippoRAG/outputs/2wikimultihopqa/openie_results_ner_gpt-4o-mini.json


In [3]:
# ============================================================
# 3. Utility Functions
# ============================================================
"""
데이터 로드, Gold 문서 추출 등 유틸리티 함수
"""

def load_json(path: str) -> Any:
    """JSON 파일 로드"""
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)


def save_json(data: Any, path: str) -> None:
    """JSON 파일 저장"""
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)


def get_gold_docs(samples: List[Dict], dataset_name: str = None) -> List[List[str]]:
    """
    샘플에서 정답 문서(supporting facts) 추출
    
    Args:
        samples: 데이터셋 샘플 리스트
        dataset_name: 데이터셋 이름 (hotpotqa는 특수 처리)
    
    Returns:
        각 샘플별 정답 문서 리스트
    """
    gold_docs = []
    for sample in samples:
        if 'supporting_facts' in sample:
            gold_title = set([item[0] for item in sample['supporting_facts']])
            gold_title_and_content_list = [item for item in sample['context'] if item[0] in gold_title]
            if dataset_name and dataset_name.startswith('hotpotqa'):
                gold_doc = [item[0] + '\n' + ''.join(item[1]) for item in gold_title_and_content_list]
            else:
                gold_doc = [item[0] + '\n' + ' '.join(item[1]) for item in gold_title_and_content_list]
        elif 'contexts' in sample:
            gold_doc = [item['title'] + '\n' + item['text'] for item in sample['contexts'] if item['is_supporting']]
        else:
            assert 'paragraphs' in sample
            gold_paragraphs = []
            for item in sample['paragraphs']:
                if 'is_supporting' in item and item['is_supporting'] is False:
                    continue
                gold_paragraphs.append(item)
            gold_doc = [
                item['title'] + '\n' + (item['text'] if 'text' in item else item['paragraph_text']) 
                for item in gold_paragraphs
            ]
        gold_doc = list(set(gold_doc))
        gold_docs.append(gold_doc)
    return gold_docs


def get_gold_answers(samples: List[Dict]) -> List[Set[str]]:
    """샘플에서 정답 추출"""
    gold_answers = []
    for sample in samples:
        gold_ans = None
        if 'answer' in sample or 'gold_ans' in sample:
            gold_ans = sample.get('answer') or sample.get('gold_ans')
        elif 'reference' in sample:
            gold_ans = sample['reference']
        elif 'obj' in sample:
            gold_ans = set([sample['obj']] + [sample.get('possible_answers', '')] + 
                          [sample.get('o_wiki_title', '')] + [sample.get('o_aliases', '')])
            gold_ans = list(gold_ans)
        
        assert gold_ans is not None
        if isinstance(gold_ans, str):
            gold_ans = [gold_ans]
        gold_ans = set(gold_ans)
        if 'answer_aliases' in sample:
            gold_ans.update(sample['answer_aliases'])
        gold_answers.append(gold_ans)
    return gold_answers


def collect_paragraphs_for_step(
    samples: List[Dict], 
    corpus: List[Dict], 
    step: int,
    dataset_name: str
) -> List[str]:
    """
    특정 Step까지의 샘플에서 필요한 문서들 수집
    
    Args:
        samples: 전체 샘플 리스트
        corpus: 전체 코퍼스
        step: 수집할 step 수 (처음 N개 샘플)
        dataset_name: 데이터셋 이름
    
    Returns:
        문서 리스트 (title + text 형식)
    """
    step_samples = samples[:step]
    needed_paragraphs = set()
    
    for sample in step_samples:
        if 'paragraphs' in sample:
            for para in sample['paragraphs']:
                text = para.get('paragraph_text') or para.get('text', '')
                if text:
                    needed_paragraphs.add(text)
        elif 'context' in sample:
            for ctx in sample['context']:
                if isinstance(ctx, list) and len(ctx) >= 2:
                    title = ctx[0]
                    if dataset_name.startswith('hotpotqa'):
                        text = ''.join(ctx[1])
                    else:
                        text = ' '.join(ctx[1]) if isinstance(ctx[1], list) else ctx[1]
                    needed_paragraphs.add(text)
    
    # corpus에서 해당 문서들 수집
    docs = []
    for doc in corpus:
        if doc.get('text') in needed_paragraphs:
            docs.append(f"{doc['title']}\n{doc['text']}")
    
    return list(set(docs))


def compute_doc_hash(doc: str, prefix: str = "chunk-") -> str:
    """문서의 해시값 계산"""
    doc_hash = hashlib.md5(doc.encode('utf-8')).hexdigest()
    return f"{prefix}{doc_hash}"


In [4]:
# ============================================================
# 4. OpenIE File Generation Functions
# ============================================================
"""
Step별 OpenIE 파일 생성 함수
- 전체 OpenIE 결과에서 해당 Step에 필요한 문서만 추출하여 저장
"""

def create_step_openie_files(
    config: DatasetConfig,
    samples: List[Dict],
    corpus: List[Dict],
    openie_data: Dict,
    steps: List[int] = None,
    llm_name: str = None
) -> Dict[int, str]:
    """
    Step별 OpenIE 파일 생성
    
    Args:
        config: 데이터셋 설정
        samples: 전체 샘플 리스트
        corpus: 전체 코퍼스
        openie_data: 전체 OpenIE 결과
        steps: Step 리스트
        llm_name: LLM 이름 (파일명용)
    
    Returns:
        {step: openie_file_path} 딕셔너리
    """
    steps = steps or STEPS
    llm_name = llm_name or LLM_NAME
    
    print(f"\n{'='*60}")
    print(f"[{config.name}] Step별 OpenIE 파일 생성")
    print(f"{'='*60}")
    
    # OpenIE 결과를 idx로 인덱싱
    openie_docs = openie_data.get('docs', [])
    openie_by_idx = {}
    openie_by_passage = {}
    for doc in openie_docs:
        idx = doc.get('idx', '')
        passage = doc.get('passage', '')
        if idx:
            openie_by_idx[idx] = doc
        if passage:
            openie_by_passage[passage] = doc
    
    print(f"  전체 OpenIE 문서 수: {len(openie_docs)}")
    
    step_openie_paths = {}
    prev_docs = set()
    
    for step in steps:
        print(f"\n--- Step {step} ---")
        
        # 해당 Step까지의 문서 수집
        step_docs = collect_paragraphs_for_step(samples, corpus, step, config.name)
        print(f"  수집된 문서 수: {len(step_docs)}")
        
        # OpenIE 결과 필터링
        step_openie_results = []
        found_by_hash = 0
        found_by_passage = 0
        missing_docs = []
        
        for doc in step_docs:
            doc_hash = compute_doc_hash(doc)
            
            if doc_hash in openie_by_idx:
                step_openie_results.append(openie_by_idx[doc_hash])
                found_by_hash += 1
            elif doc in openie_by_passage:
                step_openie_results.append(openie_by_passage[doc])
                found_by_passage += 1
            else:
                missing_docs.append(doc[:100] + "...")
        
        print(f"  찾은 OpenIE 결과: {len(step_openie_results)} (hash: {found_by_hash}, passage: {found_by_passage})")
        print(f"  누락된 문서 수: {len(missing_docs)}")
        
        if missing_docs and len(missing_docs) <= 3:
            for m in missing_docs:
                print(f"    - {m}")
        
        # Step 디렉토리 생성 및 저장
        step_dir = f"{config.output_dir}/step_{step}"
        os.makedirs(step_dir, exist_ok=True)
        
        openie_filename = f"openie_results_ner_{llm_name.replace('/', '_')}.json"
        openie_dst_path = os.path.join(step_dir, openie_filename)
        
        openie_save = {
            "docs": step_openie_results,
            "avg_ent_chars": openie_data.get("avg_ent_chars", 0),
            "avg_ent_words": openie_data.get("avg_ent_words", 0)
        }
        
        save_json(openie_save, openie_dst_path)
        step_openie_paths[step] = openie_dst_path
        
        # 통계
        total_entities = sum(len(doc.get('extracted_entities', [])) for doc in step_openie_results)
        total_triples = sum(len(doc.get('extracted_triples', [])) for doc in step_openie_results)
        
        print(f"  ✓ 저장 완료: {openie_dst_path}")
        print(f"    - 문서: {len(step_openie_results)}, 엔티티: {total_entities}, 트리플: {total_triples}")
        
        if prev_docs:
            new_docs = set(step_docs) - prev_docs
            print(f"    - 이전 Step 대비 추가 문서: {len(new_docs)}")
        
        prev_docs = set(step_docs)
    
    return step_openie_paths


In [5]:
# ============================================================
# 5. Graph Building Functions
# ============================================================
"""
Step별 임베딩 및 그래프 구축 함수
- Continual Learning 방식: 이전 Step의 그래프를 로드하여 확장
"""

def build_step_graphs(
    config: DatasetConfig,
    samples: List[Dict],
    corpus: List[Dict],
    steps: List[int] = None,
    llm_name: str = None,
    embedding_model_name: str = None,
    force_rebuild: bool = False
) -> Dict[int, str]:
    """
    Step별 그래프 구축 (Continual Learning 방식)
    
    Step 250: 새로 시작
    Step 500+: 이전 Step의 그래프를 로드하여 확장
    
    Args:
        config: 데이터셋 설정
        samples: 전체 샘플 리스트
        corpus: 전체 코퍼스
        steps: Step 리스트
        llm_name: LLM 이름
        embedding_model_name: 임베딩 모델 이름
        force_rebuild: 기존 그래프 무시하고 재구축
    
    Returns:
        {step: graph_dir_path} 딕셔너리
    """
    steps = steps or STEPS
    llm_name = llm_name or LLM_NAME
    embedding_model_name = embedding_model_name or EMBEDDING_MODEL_NAME
    
    print(f"\n{'='*60}")
    print(f"[{config.name}] Step별 그래프 구축")
    print(f"{'='*60}")
    
    step_graph_paths = {}
    prev_step = None
    
    for i, step in enumerate(steps):
        print(f"\n{'='*40}")
        print(f"Step {step} 시작")
        print(f"{'='*40}")
        
        # 해당 Step까지의 문서 수집
        step_docs = collect_paragraphs_for_step(samples, corpus, step, config.name)
        print(f"  인덱싱할 문서 수: {len(step_docs)}")
        
        # Step 디렉토리 설정
        step_dir = f"{config.output_dir}/step_{step}"
        os.makedirs(step_dir, exist_ok=True)
        
        # 그래프 경로 확인
        model_dir_name = f"{llm_name.replace('/', '_')}_{embedding_model_name.replace('/', '_')}"
        graph_path = os.path.join(step_dir, model_dir_name, "graph.pickle")
        
        # 이미 그래프가 있으면 스킵 (force_rebuild=False일 때)
        if os.path.exists(graph_path) and not force_rebuild:
            print(f"  ✓ 기존 그래프 존재, 스킵: {graph_path}")
            step_graph_paths[step] = step_dir
            prev_step = step
            continue
        
        # HippoRAG 설정
        base_config = BaseConfig()
        base_config.save_dir = step_dir
        base_config.force_index_from_scratch = False
        base_config.force_openie_from_scratch = False
        base_config.embedding_batch_size = 16
        
        hipporag = HippoRAG(
            global_config=base_config,
            save_dir=step_dir,
            llm_model_name=llm_name,
            embedding_model_name=embedding_model_name
        )
        
        # Continual Learning: 이전 Step 그래프 로드 (첫 Step 제외)
        if i > 0 and prev_step is not None:
            prev_graph_path = os.path.join(
                f"{config.output_dir}/step_{prev_step}",
                model_dir_name,
                "graph.pickle"
            )
            
            if os.path.exists(prev_graph_path):
                print(f"  이전 그래프 로드 중: {prev_graph_path}")
                try:
                    with open(prev_graph_path, 'rb') as f:
                        hipporag.graph = pickle.load(f)
                    print(f"  ✓ 이전 그래프 로드 완료")
                    print(f"    - 노드 수: {hipporag.graph.vcount()}")
                    print(f"    - 엣지 수: {hipporag.graph.ecount()}")
                except Exception as e:
                    print(f"  ⚠ 이전 그래프 로드 실패: {e}")
                    print(f"  → 새로 시작")
            else:
                print(f"  ⚠ 이전 그래프 없음, 새로 시작")
        else:
            print(f"  첫 Step, 새로 시작")
        
        # 인덱싱 실행
        print(f"  인덱싱 시작...")
        hipporag.index(step_docs)
        
        # 그래프 정보 출력
        if hasattr(hipporag, 'graph') and hipporag.graph is not None:
            print(f"  ✓ 인덱싱 완료")
            print(f"    - 노드 수: {hipporag.graph.vcount()}")
            print(f"    - 엣지 수: {hipporag.graph.ecount()}")
        
        step_graph_paths[step] = step_dir
        prev_step = step
        
        # 메모리 정리
        del hipporag
        gc.collect()
        torch.cuda.empty_cache()
    
    return step_graph_paths


In [ ]:
# ============================================================
# 6. Main Pipeline Function
# ============================================================
"""
전체 파이프라인 실행 함수
1. OpenIE 파일 생성
2. Step별 그래프 구축
"""

def run_dataset_pipeline(
    dataset_name: str,
    steps: List[int] = None,
    force_rebuild: bool = False,
    skip_openie: bool = False,
    skip_graph: bool = False
) -> Dict[str, Any]:
    """
    데이터셋에 대한 전체 파이프라인 실행
    
    Args:
        dataset_name: 데이터셋 이름 (hotpotqa, 2wikimultihopqa)
        steps: Step 리스트
        force_rebuild: 기존 그래프 무시하고 재구축
        skip_openie: OpenIE 파일 생성 스킵
        skip_graph: 그래프 구축 스킵
    
    Returns:
        결과 딕셔너리
    """
    steps = steps or STEPS
    
    if dataset_name not in DATASETS:
        raise ValueError(f"Unknown dataset: {dataset_name}. Available: {list(DATASETS.keys())}")
    
    config = DATASETS[dataset_name]
    
    print(f"\n{'#'*60}")
    print(f"# Dataset: {dataset_name}")
    print(f"# Steps: {steps}")
    print(f"{'#'*60}")
    
    # 1. 데이터 로드
    print(f"\n[1] 데이터 로드 중...")
    
    if not os.path.exists(config.samples_path):
        print(f"  ⚠ 샘플 파일 없음: {config.samples_path}")
        return {"error": f"Samples file not found: {config.samples_path}"}
    
    if not os.path.exists(config.corpus_path):
        print(f"  ⚠ 코퍼스 파일 없음: {config.corpus_path}")
        return {"error": f"Corpus file not found: {config.corpus_path}"}
    
    samples = load_json(config.samples_path)
    corpus = load_json(config.corpus_path)
    
    print(f"  ✓ 샘플 수: {len(samples)}")
    print(f"  ✓ 코퍼스 문서 수: {len(corpus)}")
    
    # 2. OpenIE 데이터 로드 및 파일 생성
    openie_paths = {}
    if not skip_openie:
        print(f"\n[2] OpenIE 파일 생성 중...")
        
        if not os.path.exists(config.openie_path):
            print(f"  ⚠ OpenIE 파일 없음: {config.openie_path}")
            print(f"  → OpenIE 파일 생성 스킵")
        else:
            openie_data = load_json(config.openie_path)
            print(f"  ✓ OpenIE 데이터 로드 완료: {len(openie_data.get('docs', []))} 문서")
            
            openie_paths = create_step_openie_files(
                config=config,
                samples=samples,
                corpus=corpus,
                openie_data=openie_data,
                steps=steps
            )
    else:
        print(f"\n[2] OpenIE 파일 생성 스킵")
    
    # 3. 그래프 구축
    graph_paths = {}
    if not skip_graph:
        print(f"\n[3] 그래프 구축 중...")
        
        graph_paths = build_step_graphs(
            config=config,
            samples=samples,
            corpus=corpus,
            steps=steps,
            force_rebuild=force_rebuild
        )
    else:
        print(f"\n[3] 그래프 구축 스킵")
    
    # 결과 요약
    print(f"\n{'='*60}")
    print(f"[{dataset_name}] 완료!")
    print(f"{'='*60}")
    
    result = {
        "dataset": dataset_name,
        "steps": steps,
        "openie_paths": openie_paths,
        "graph_paths": graph_paths,
        "output_dir": config.output_dir,
    }
    
    return result


def run_all_datasets(
    steps: List[int] = None,
    force_rebuild: bool = False,
    skip_openie: bool = False,
    skip_graph: bool = False
) -> Dict[str, Any]:
    """
    모든 데이터셋에 대해 파이프라인 실행
    
    Returns:
        {dataset_name: result} 딕셔너리
    """
    results = {}
    
    for dataset_name in DATASETS.keys():
        result = run_dataset_pipeline(
            dataset_name=dataset_name,
            steps=steps,
            force_rebuild=force_rebuild,
            skip_openie=skip_openie,
            skip_graph=skip_graph
        )
        results[dataset_name] = result
    
    return results


✓ 메인 파이프라인 함수 정의 완료


---

# 실행 섹션

## 1. HotpotQA 데이터셋

In [ ]:
# ============================================================
# 7. Run HotpotQA Pipeline
# ============================================================
"""
HotpotQA 데이터셋 파이프라인 실행

출력 경로: _hippo_rag_MHQA_CL/hotpotqa/step_{250,500,750,1000}/
"""

# HotpotQA 실행
hotpotqa_result = run_dataset_pipeline(
    dataset_name="hotpotqa",
    steps=STEPS,
    force_rebuild=False,  # True로 하면 기존 그래프 무시하고 재구축
    skip_openie=False,    # OpenIE 파일 이미 있으면 True
    skip_graph=False      # 그래프만 스킵하려면 True
)

print("\n결과:")
print(f"  Output Dir: {hotpotqa_result.get('output_dir')}")

## 2. 2WikiMultiHopQA 데이터셋

In [ ]:
# ============================================================
# 8. Run 2WikiMultiHopQA Pipeline
# ============================================================
"""
2WikiMultiHopQA 데이터셋 파이프라인 실행

출력 경로: _hippo_rag_MHQA_CL/2wikimultihopqa/step_{250,500,750,1000}/
"""

# 2WikiMultiHopQA 실행
wiki2_result = run_dataset_pipeline(
    dataset_name="2wikimultihopqa",
    steps=STEPS,
    force_rebuild=False,
    skip_openie=False,
    skip_graph=False
)

print("\n결과:")
print(f"  Output Dir: {wiki2_result.get('output_dir')}")